### Etude des pistes cyclables

In [14]:
!pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   --------- ----------------------------- 30.7/125.4 kB 325.1 kB/s eta 0:00:01
   ---------------------- ---------------- 71.7/125.4 kB 563.7 kB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 817.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [16]:
import pandas as pd
import json
from geopy.distance import geodesic

In [8]:
df = pd.read_csv('pistes_cyclables.csv')

In [10]:
df.head()

,ogc_fid,geo_point_2d,anne_maj,lametro,type,name,geo_shape
0,1489,"45.17402,5.74223",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.74223,4..."
1,1490,"45.17272,5.73701",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73701,4..."
2,1491,"45.17273,5.73488",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73488,4..."
3,1492,"45.17315,5.73476",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73476,4..."
4,1625,"45.20183,5.75957",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.75957,4..."


### Vérification des type LineString

In [22]:
# Fonction pour vérifier le type de géométrie
def is_linestring(geo_shape):
    try:
        shape = json.loads(geo_shape)
        return shape['type'] == 'LineString'
    except:
        return False

# Vérifier si toutes les géométries sont des LineString
df['is_linestring'] = df['geo_shape'].apply(is_linestring)

# Résumé des résultats
print("Résumé des types de géométries :")
print(df['is_linestring'].value_counts())

# Identifier les géométries non conformes, s'il y en a
non_linestring_rows = df[~df['is_linestring']]
if not non_linestring_rows.empty:
    print("Lignes avec des géométries non conformes :")
    print(non_linestring_rows)
else:
    print("Toutes les géométries sont des LineString.")

Résumé des types de géométries :
is_linestring
True    1202
Name: count, dtype: int64
Toutes les géométries sont des LineString.


### Calcul des distances de la piste cyclable

In [18]:
# Fonction pour calculer la distance d'une piste cyclable à partir de la colonne geo_shape
def calculate_distance(geo_shape):
    try:
        # Charger les coordonnées GeoJSON
        shape = json.loads(geo_shape)
        if shape['type'] == 'LineString':
            coordinates = shape['coordinates']
            # Calculer la distance totale entre les points consécutifs
            distances = [
                geodesic(coordinates[i], coordinates[i+1]).kilometers
                for i in range(len(coordinates) - 1)
            ]
            return sum(distances)
    except:
        return None  # Retourner None si un problème survient

# Ajouter une colonne pour les distances
df['distance_km'] = df['geo_shape'].apply(calculate_distance)

# Enregistrer le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
df.to_csv('fichier_avec_distances.csv', index=False)

# Afficher les premières lignes pour vérifier
print(df.head())


   ogc_fid      geo_point_2d  anne_maj  lametro        type name  \
0     1489  45.17402,5.74223    2016.0        1  chronovelo  NaN   
1     1490  45.17272,5.73701    2016.0        1  chronovelo  NaN   
2     1491  45.17273,5.73488    2016.0        1  chronovelo  NaN   
3     1492  45.17315,5.73476    2016.0        1  chronovelo  NaN   
4     1625  45.20183,5.75957    2016.0        1  chronovelo  NaN   

                                           geo_shape  distance_km  
0  {"type":"LineString","coordinates":[[5.74223,4...     0.231531  
1  {"type":"LineString","coordinates":[[5.73701,4...     0.235551  
2  {"type":"LineString","coordinates":[[5.73488,4...     0.055292  
3  {"type":"LineString","coordinates":[[5.73476,4...     0.106301  
4  {"type":"LineString","coordinates":[[5.75957,4...     0.234373  
